### ✅ 1단계: 문서 분할 설정

In [ ]:
#pip install -U langchain-openai

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

sk


### ✅ 2단계: 임베딩 모델 설정

In [2]:
from langchain_openai import OpenAIEmbeddings
import numpy as np

# OpenAI의 "text-embedding-3-small" 모델을 사용하여 임베딩 객체 생성
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
print("✅ 임베딩 객체가 성공적으로 생성되었습니다.")

✅ 임베딩 객체가 성공적으로 생성되었습니다.


In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

print("[텍스트 분할기 설정]")
print("="*50)

# 텍스트 분할 설정 (chunk_size, chunk_overlap 증가로 속도 개선 의도)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=3500,      # 법률 사례는 1200~1800자 권장하지만 속도 위해 늘림
    chunk_overlap=300,    # 맥락 보존을 위해 200~400자 설정 유지
    separators=[
        "\n【사건개요】", "\n【쟁점사항】", "\n【처리경위】", "\n【처리결과】",
        "\n■", "\n\n", "\n", ".", " ", ""
    ]
)

# 설정 값 출력
print(f"✅ 청크 크기(chunk_size): {text_splitter._chunk_size}자")
print(f"✅ 오버랩(chunk_overlap): {text_splitter._chunk_overlap}자")
print(f"✅ 분할 구분자(separators): {text_splitter._separators[:4]}... (총 {len(text_splitter._separators)}개)")
print("="*50)

# 테스트 문서 분할 (가상의 예시 문서 - 길게 이어붙여서 청크 분할 효과 확인)
sample_text = """
【사건개요】 
게임사 A는 2023년 5월 출시한 모바일 게임 내 프리미엄 아이템 확률 표시를 누락하여 이용자 1,200명이 집단 분쟁 제기.

【쟁점사항】
1. 확률 표시 의무 위반 여부
2. 피해 금액 산정 기준
3. 환불 범위 설정 문제

【처리결과】
게임사는 전체 이용자에게 아이템 비용 환불 및 10% 추가 보상 제공.

【사건개요】 
또 다른 게임사 B는 2024년 1월 출시한 게임에서 미성년자 결제 관련 분쟁이 발생함.

【쟁점사항】
1. 부모 동의 없는 결제 문제
2. 환불 정책의 적절성
3. 게임사 책임 범위

【처리경위】
게임사는 피해자들에게 개별 연락하여 환불 진행 중이며, 법적 조치도 검토함.

【처리결과】
일부 환불 완료, 추가 보상 여부는 추후 결정 예정.
"""

print("\n[테스트 분할 실행]")
print("="*50)
try:
    chunks = text_splitter.split_text(sample_text)
    print(f"✅ 분할 완료 (생성된 청크 수: {len(chunks)}개)")
    print("\n[첫 번째 청크 내용]")
    print("-"*30)
    print(chunks[0])
    print("-"*30)
    if len(chunks) > 1:
        print("\n[두 번째 청크 내용]")
        print("-"*30)
        print(chunks[1])
        print("-"*30)
except Exception as e:
    print(f"❌ 분할 실패: {str(e)}")


[텍스트 분할기 설정]
✅ 청크 크기(chunk_size): 3500자
✅ 오버랩(chunk_overlap): 300자
✅ 분할 구분자(separators): ['\n【사건개요】', '\n【쟁점사항】', '\n【처리경위】', '\n【처리결과】']... (총 10개)

[테스트 분할 실행]
✅ 분할 완료 (생성된 청크 수: 1개)

[첫 번째 청크 내용]
------------------------------
【사건개요】 
게임사 A는 2023년 5월 출시한 모바일 게임 내 프리미엄 아이템 확률 표시를 누락하여 이용자 1,200명이 집단 분쟁 제기.

【쟁점사항】
1. 확률 표시 의무 위반 여부
2. 피해 금액 산정 기준
3. 환불 범위 설정 문제

【처리결과】
게임사는 전체 이용자에게 아이템 비용 환불 및 10% 추가 보상 제공.

【사건개요】 
또 다른 게임사 B는 2024년 1월 출시한 게임에서 미성년자 결제 관련 분쟁이 발생함.

【쟁점사항】
1. 부모 동의 없는 결제 문제
2. 환불 정책의 적절성
3. 게임사 책임 범위

【처리경위】
게임사는 피해자들에게 개별 연락하여 환불 진행 중이며, 법적 조치도 검토함.

【처리결과】
일부 환불 완료, 추가 보상 여부는 추후 결정 예정.
------------------------------


In [4]:
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS

# PDF 파일 경로
pdf_path = "C:/mylangchain/langchain_basic/data/콘텐츠분쟁해결_사례.pdf"

# 1. 로딩
loader = PyPDFLoader(pdf_path)
pages = loader.load()

# 2. 분할
documents = text_splitter.split_documents(pages)

# 3. 벡터 임베딩 & 저장소 생성
vectorstore = FAISS.from_documents(documents, embedding=embeddings)

# 4. 검색기 생성
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}
)

print(f"총 분할 문서 수: {len(documents)}개")


총 분할 문서 수: 102개


In [10]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0.2,
    max_tokens=2000
)

print("✅ ChatOpenAI LLM(gpt-4o) 객체 생성 완료")


✅ ChatOpenAI LLM(gpt-4o) 객체 생성 완료


In [11]:
from langchain.prompts import PromptTemplate

# 프롬프트 템플릿 문자열
prompt_template = """
당신은 콘텐츠 분야 전문 법률 자문사입니다. 
아래 분쟁조정 사례들을 바탕으로 정확하고 전문적인 법률 조언을 제공해주세요.

관련 분쟁사례:
{context}

상담 내용: {question}

답변 가이드라인:
1. 제시된 사례들을 근거로 답변하세요
2. 관련 법령이나 조항이 있다면 명시하세요
3. 실무적 해결방안을 단계별로 제시하세요
4. 비슷한 사례의 처리경위와 결과를 참고하여 설명하세요
5. 사례에 없는 내용은 "제시된 사례집에서는 확인할 수 없습니다"라고 명시하세요

전문 법률 조언:
"""

# 템플릿 객체 생성
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template
)

print("✅ 프롬프트 템플릿 객체 생성 완료")


✅ 프롬프트 템플릿 객체 생성 완료


In [12]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True
)

print("QA 체인 생성 완료 ✅")


QA 체인 생성 완료 ✅


In [13]:
test_questions = [
    "온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?",
    "인터넷 강의를 중도 해지하려고 하는데 과도한 위약금을 요구받고 있습니다. 정당한가요?",
    "무료체험 후 자동으로 유료전환되어 요금이 청구되었습니다. 환불 가능한가요?",
    "미성년자가 부모 동의 없이 게임 아이템을 구매했습니다. 환불받을 수 있는 방법이 있나요?",
    "온라인 교육 서비스가 광고와 다르게 제공되어 계약을 해지하고 싶습니다. 가능한가요?"
]

for i, q in enumerate(test_questions, 1):
    print(f"\n\n========================= 질문 {i} =========================")
    print(f"❓ 질문: {q}")

    # QA 체인 실행
    result = qa_chain.invoke({"query": q})

    # 답변 출력 (한 줄씩)
    print("\n✅ 생성된 답변:")
    for line in result["result"].split('\n'):
        print(line.strip())

    # 참조된 문서 정보 출력 (한 줄씩)
    print("\n📚 참고 문서 목록:")
    for j, doc in enumerate(result["source_documents"], 1):
        source = doc.metadata.get("source", "출처 정보 없음")
        preview = doc.page_content[:200].replace('\n', ' ').strip()
        print(f"  {j}. 문서 출처: {source}")
        print(f"     - 내용 미리보기: {preview} ...")




========================= 질문 1 =========================
❓ 질문: 온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?

✅ 생성된 답변:
온라인 게임에서 시스템 오류로 인해 아이템이 사라진 경우, 게임회사가 복구를 거부하는 상황에 대한 법률 조언을 다음과 같이 제공합니다.

1. **사례 분석**:
- 제시된 사례들에서 공통적으로 나타나는 문제는 시스템 오류로 인한 아이템 소실에 대한 복구 요청입니다. 특히, 사례 1-가-1(2006)과 1-가-1(2009)에서는 시스템 오류로 인한 아이템 소실에 대해 게임회사가 복구를 거부한 상황이 있습니다.
- 2006년 사례에서는 계정 명의자가 아닌 경우 복구가 불가하다는 게임사의 약관에 따라 복구가 거부되었습니다.
- 2009년 사례에서는 시스템 오류가 아닌 사용자 측의 문제일 가능성이 있다는 이유로 복구가 거부되었습니다.

2. **관련 법령 및 조항**:
- 약관의 규제에 관한 법률 제11조는 고객의 권익 보호를 규정하고 있으며, 소비자분쟁해결기준(공정위고시 제2010-1호)에서는 전자상거래 등에서의 소비자 보호에 관한 기준을 제시하고 있습니다.

3. **실무적 해결방안**:
- **1단계: 증거 수집**: 시스템 오류로 인한 아이템 소실을 증명할 수 있는 증거(스크린샷, 로그 기록 등)를 수집합니다.
- **2단계: 게임사와의 협의**: 수집한 증거를 바탕으로 게임사에 다시 복구 요청을 하며, 게임사의 약관 및 운영정책을 검토하여 복구 가능성을 확인합니다.
- **3단계: 소비자 보호 기관에 문의**: 게임사가 복구를 거부할 경우, 한국소비자원이나 전자거래분쟁조정위원회에 중재를 요청할 수 있습니다.
- **4단계: 법적 조치 고려**: 위의 단계들이 모두 실패할 경우, 법적 조치를 고려할 수 있으며, 이때는 변호사와 상담하여 소송 가능성을 검토합니다.

4. **비슷한 사례의 처리경위와 결과**:
-

In [18]:
def classify_dispute_type(query):
    game_keywords = [
        "게임", "아이템", "계정", "캐릭터", "레벨", "길드", "온라인게임",
        "접속", "패치", "서버", "버그", "결제", "인벤토리", "퀘스트",
        "보상", "랭킹", "게임머니", "게임사", "멀티플레이", "클래스",
        "업데이트", "이벤트", "친구", "길드전", "게임내재화"
    ]

    elearning_keywords = [
        "강의", "온라인교육", "이러닝", "수강", "환불", "화상교육",
        "수업", "강사", "학습", "콘텐츠", "교육비", "교재", "출석",
        "시험", "자격증", "학점", "플랫폼", "멤버십", "재수강",
        "과제", "퀴즈", "학습관리", "튜터링", "수강신청"
    ]

    web_keywords = [
        "웹사이트", "무료체험", "자동결제", "구독", "사이트",
        "회원가입", "탈퇴", "요금", "결제", "환불", "광고",
        "서비스", "이용약관", "개인정보", "쿠키", "프로모션",
        "업데이트", "고객센터", "접속장애", "비밀번호", "보안",
        "트래픽", "검색", "알림", "메일", "서버관리"
    ]
    
    query_lower = query.lower()
    
    if any(keyword in query_lower for keyword in game_keywords):
        return "게임"
    elif any(keyword in query_lower for keyword in elearning_keywords):
        return "이러닝"
    elif any(keyword in query_lower for keyword in web_keywords):
        return "웹콘텐츠"
    else:
        return "기타"

# 테스트 질문 리스트
test_questions = [
    "온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?",
    "인터넷 강의를 중도 해지하려고 하는데 과도한 위약금을 요구받고 있습니다. 정당한가요?",
    "무료체험 후 자동으로 유료전환되어 요금이 청구되었습니다. 환불 가능한가요?",
    "미성년자가 부모 동의 없이 게임 아이템을 구매했습니다. 환불받을 수 있는 방법이 있나요?",
    "온라인 교육 서비스가 광고와 다르게 제공되어 계약을 해지하고 싶습니다. 가능한가요?",
    "길드에서 비매너 행위로 강퇴 당했습니다. 다시 복귀 할 수 있을까요?"
]

# 분쟁 유형 출력
for i, question in enumerate(test_questions, 1):
    dispute_type = classify_dispute_type(question)
    print(f"질문 {i}: {question}")
    print(f"분쟁 유형: {dispute_type}")
    print("-" * 40)


질문 1: 온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?
분쟁 유형: 게임
----------------------------------------
질문 2: 인터넷 강의를 중도 해지하려고 하는데 과도한 위약금을 요구받고 있습니다. 정당한가요?
분쟁 유형: 이러닝
----------------------------------------
질문 3: 무료체험 후 자동으로 유료전환되어 요금이 청구되었습니다. 환불 가능한가요?
분쟁 유형: 이러닝
----------------------------------------
질문 4: 미성년자가 부모 동의 없이 게임 아이템을 구매했습니다. 환불받을 수 있는 방법이 있나요?
분쟁 유형: 게임
----------------------------------------
질문 5: 온라인 교육 서비스가 광고와 다르게 제공되어 계약을 해지하고 싶습니다. 가능한가요?
분쟁 유형: 웹콘텐츠
----------------------------------------
질문 6: 길드에서 비매너 행위로 강퇴 당했습니다. 다시 복귀 할 수 있을까요?
분쟁 유형: 게임
----------------------------------------
